# APIM ❤️ OpenAI

## Private connectivity lab
![flow](./architecture.png)

Playground to show how to create a private network for consuming LLMs from `AI Services`.
This lab demonstrates how to create a private network for consuming LLMs from `AI Services` using `Private Link Services`, `Azure API Management (APIM)`, and `Azure Front Door`.

Notes:
- `Azure OpenAI` is only accessible through `Private Endpoints`. Public network access is disabled.
- `Azure API Management` is integrated in the private network and is used to manage the traffic to the `Azure OpenAI` service through `Private Endpoints`.
- `Azure API Management` is not accessible from a public network. The only access is through `Azure Front Door`.
- `Azure Front Door` manages the traffic to the `Azure API Management` service through `Private Link Service`.
- **`Azure Front Door` is the only publicly-accessible service.**

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [11]:
import os, sys, json
sys.path.insert(1, '../src')  
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"

apim_sku = 'Standardv2'

# Prioritize East US until exhaustion (simulate PTU with TPM), then equally distribute between Sweden and West US (consumption fallback)
openai_resources = [
    {"name": "openai1", "capacity": 20, "location": "eastus2", "priority": 1},
    {"name": "openai2", "capacity": 20, "location": "westus", "priority": 2, "weight": 50}
]

openai_deployment_name = "gpt-4.1"
openai_model_name = "gpt-4.1"
openai_model_version = "2025-04-14"
openai_model_capacity = 20
openai_model_sku = 'GlobalStandard'
openai_api_version = "2024-02-01"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 12:06:43.721197 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [12]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 12:06:50.992684 :5s]
👉🏽 Current user: pablocastao@microsoft.com
👉🏽 Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
👉🏽 Subscription ID: 06d043e2-5a2e-46bf-bf48-fffee525f377


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.

In [13]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "apimPublicNetworkAccess": { "value": "Enabled" }
    }
}

# Write the parameters to the params.json file
with open('../infra/params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file ../infra/main.bicep --parameters ../infra/params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-notebooks 
👉🏽 Resource group lab-notebooks does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name lab-notebooks --location eastus2 --tags source=ai-gateway 
✅ Resource group 'lab-notebooks' created ⌚ 13:21:46.870086 :4s]
⚙️ Running: az deployment group create --name notebooks --resource-group lab-notebooks --template-file ../infra/main.bicep --parameters ../infra/params.json 
✅ Deployment 'notebooks' succeeded ⌚ 13:28:13.310408 :26s]


<a id='3'></a>
### 3️⃣ Approve Front Door private link connection to APIM

In the deployed Bicep template, Azure Front Door will establish a private link connection to the API Management service. This connection should be approved. Run the following command to approve the connection.

In [14]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_resource_id = utils.get_deployment_output(output, 'apimResourceId', 'apimResourceId')

    outputPls = utils.run(f"az network private-endpoint-connection list --id {apim_resource_id} --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv")
    
    if outputPls.success:
        pls_connection_id = outputPls.text
        print(pls_connection_id)
        utils.run(f"az network private-endpoint-connection approve --id {pls_connection_id} --description 'Approved'", f"Private Link Connection approved.", f"Failed to approve Private Link Connection: {pls_connection_id}")

⚙️ Running: az deployment group show --name notebooks -g lab-notebooks 
✅ Retrieved deployment: notebooks ⌚ 13:36:33.853751 :5s]
👉🏽 apimResourceId: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-notebooks/providers/Microsoft.ApiManagement/service/apim-orileqqs2tzlo
⚙️ Running: az network private-endpoint-connection list --id /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-notebooks/providers/Microsoft.ApiManagement/service/apim-orileqqs2tzlo --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv 
/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-notebooks/providers/Microsoft.ApiManagement/service/apim-orileqqs2tzlo/privateEndpointConnections/2000e631-2052-4c70-976a-cc29c02df1a9

⚙️ Running: az network private-endpoint-connection approve --id /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-notebooks/providers/Microsoft.ApiManagement/service/apim-orileqqs2tzlo/pri

<a id='4'></a>
### 4️⃣ Disabling APIM public network access

As of May 2025, during the creation, the `APIM` service cannot disable the public network access. This behavior might change in the future. As a workaround, you can disable the public network access after the deployment is completed. To do that, you can run the Bicep template again after changing the `apimPublicNetworkAccess` parameter to `Disabled`. This will update the APIM service and disable the public network access.

In [17]:
bicep_parameters['parameters']['apimPublicNetworkAccess']['value'] = "Disabled"

# Write the parameters to the params.json file
with open('../infra/params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file ../infra/main.bicep --parameters ../infra/params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az deployment group create --name notebooks --resource-group lab-notebooks --template-file ../infra/main.bicep --parameters ../infra/params.json 
✅ Deployment 'notebooks' succeeded ⌚ 13:54:51.379791 :10s]


<a id='5'></a>
### 5️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [18]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    frontdoor_endpoint = utils.get_deployment_output(output, 'frontDoorEndpointHostName', 'Front Door Endpoint')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)

⚙️ Running: az deployment group show --name notebooks -g lab-notebooks 
✅ Retrieved deployment: notebooks ⌚ 13:57:26.980523 :7s]
👉🏽 Front Door Endpoint: afd-rkt5yhtbkebnw-hdbubngvbjaxgdbz.b01.azurefd.net
👉🏽 APIM API Gateway URL: https://apim-orileqqs2tzlo.azure-api.net
👉🏽 APIM Subscription Key (masked): ****ebc0


<a id='6'></a>
### 6️⃣ 🧪 Test the API using a direct HTTP call through Frontdoor

Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

In [25]:
import requests

def call_api(resource_url):
    url = f"{resource_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"

    messages = {"messages": [
        {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)

    print(f"Response status code: {response.status_code}")

    if (response.status_code == 200):
        data = json.loads(response.text)
        print(f"💬 {data.get('choices')[0].get('message').get('content')}\n")
    else:
        print(f"{response.text}\n")

call_api(f"https://{frontdoor_endpoint}")

Response status code: 404
<!DOCTYPE html>
<!-- saved from url=(0058)https://df.onecloud.azure-test.net/Error/UE_404?shown=true -->
<html lang="en" dir="ltr">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>Page not found</title>

    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="msapplication-config" content="none" />
    <link rel="icon" type="image/x-icon" href="" />
    <link rel="shortcut icon" type="image/x-icon" href="" />
    <meta name="robots" content="noindex, nofollow" />

    <style type="text/css">

body {
    margin: 0;
    padding: 0;
    color: #fff;
    font-family: wf_segoe-ui_normal,"Segoe UI","Segoe WP",Tahoma,Arial,sans-serif;
    font-weight: 400;
    background: #babcbe url("") top left repeat;
}

img {
    border: 0;
}

ul {
    list-style: none;
    margin: 0;
    padding: 0;
 padding-top: 30px;
}

    ul li {
        margin-bottom: 24px;
        font-family: wf_segoe-ui_normal,"Segoe UI","

In [26]:
frontdoor_endpoint

'afd-rkt5yhtbkebnw-hdbubngvbjaxgdbz.b01.azurefd.net'

<a id='7'></a>
### 7️⃣ 🧪 Test APIM API access through public network

APIM is not accessible from public network. The only way to access it is through Azure Front Door. This test should fail with a 403 error.

In [20]:
call_api(apim_resource_gateway_url)

Response status code: 403
{ "statusCode": 403, "message": "Request originated from client public IP address 201.26.88.190, public network access on this `Microsoft.ApiManagement/service/apim-orileqqs2tzlo` is disabled. To connect to `Microsoft.ApiManagement/service/apim-orileqqs2tzlo`, please use the Private Endpoint from inside your virtual network. To learn more https://aka.ms/apim-privateendpoint " }



<a id='8'></a>
### 8 🧪 Test APIM API access through private virtual network

APIM is accessible in the private virtual network through a `Private Endpoint`. This means that any resource within this network can communicate securely with the APIM service. To validate this behavior, the Bicep template creates an `Azure virtual machine` that will act as a `jumpbox`. This VM is deployed in the same virtual network as the APIM service and has access to it.
The following code will generate a `cUrl` command that will test the API access. This script will be executed in the jumpbox vm through the `az vm run-command invoke` command.

Let's now run the script in the jumpbox VM to test the API access through the private virtual network. The command will be executed in the jumpbox VM and will return the response from the APIM service.

In [21]:
import subprocess
import json
import re

command = (f"az vm run-command invoke "
           f"-g {resource_group_name} "
           f"-n vm-jumpbox "
           f"--query value[].message "
           f"-o tsv "
           f"--command-id RunShellScript "
           f"--scripts \"curl -X POST '{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}' "
           f"-H 'Content-Type: application/json' "
           f"-H 'api-key: {apim_subscription_key}' "
           f"-d '{{\\\"messages\\\": [  {{\\\"role\\\": \\\"system\\\", \\\"content\\\": \\\"You are a helpful assistant.\\\"}},   {{\\\"role\\\": \\\"user\\\", \\\"content\\\": \\\"What are 3 things to visit in Seattle?\\\"}}      ]}}'\""
           )

# Execute the command and capture its output
result = subprocess.run(command, shell=True, capture_output=True, text=True)
output_text = result.stdout

# Extract just the JSON portion from the output
json_match = re.search(r'(\{.*"choices".*\})', output_text)

if json_match:
    try:
        # Parse the JSON response
        openai_response = json.loads(json_match.group(1))
        
        # Extract the assistant's message content
        assistant_message = openai_response['choices'][0]['message']['content']
        
        # Display the extracted message content in a readable format
        print("\n🤖 Assistant's response:\n")
        print(assistant_message)
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON: {e}")
else:
    print("Could not find JSON response in the output")
    print("Raw output:", output_text)


🤖 Assistant's response:

Sure! Here are three great things to visit in Seattle:

1. **Space Needle** â€“ This iconic landmark offers stunning views of the city, Mount Rainier, and Puget Sound from its observation deck.

2. **Pike Place Market** â€“ A historic public market where youâ€™ll find fresh seafood, local produce, unique shops, and the famous flying fish.

3. **Chihuly Garden and Glass** â€“ An impressive exhibition showcasing the colorful glass sculptures of artist Dale Chihuly, located right next to the Space Needle.

Enjoy your trip!


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.